In [11]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from KoBERT.kobert.utils import get_tokenizer
from KoBERT.kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

import pandas as pd
from sklearn.model_selection import train_test_split

##GPU 사용 시
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [12]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=NUM_CLASS,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [20]:
model = BERTClassifier(bertmodel,  dr_rate=0.5)

In [21]:
model.load_state_dict(torch.load("models/kobert.pth"))

<All keys matched successfully>

In [22]:
model.to(device)

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [23]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_key, label_key, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i]) for i in dataset[sent_key]]
        
        if not label_key == None:
            self.mode = "train"
        else:
            self.mode = "test"
            
        if self.mode == "train":
            self.labels = [np.int32(i) for i in dataset[label_key]]
        else:
            self.labels = [np.int32(0) for i in dataset[sent_key]]

    def __getitem__(self, i):
        if self.mode == "train":
            return (self.sentences[i] + (self.labels[i], ))
        else:
            return self.sentences[i]

    def __len__(self):
        return (len(self.labels))

using cached model


In [24]:
max_len = 32
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
NUM_CLASS = 7

In [25]:
dataset_test = pd.read_csv("data/test_data.csv",index_col=False)
data_test = BERTDataset(dataset_test, "title", None, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

submission = pd.read_csv('data/sample_submission.csv')

In [31]:
model.eval()
pred = []

for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    out = model(token_ids, valid_length, segment_ids)
    _,max_indices = torch.max(out,1)
    for idx in max_indices.cpu().numpy():
        pred.append(idx)


/home/park/Workspace/dacon-new-classification/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/286 [00:00<?, ?it/s]

[2, 3, 2, 2, 3, 0, 5, 3, 4, 4, 4, 6, 4, 5, 6, 1, 6, 2, 4, 4, 4, 4, 4, 0, 0, 3, 6, 0, 5, 1, 3, 1, 4, 5, 2, 4, 5, 4, 6, 5, 5, 5, 5, 3, 0, 5, 3, 6, 1, 6, 0, 4, 6, 4, 5, 2, 4, 6, 3, 3, 4, 4, 4, 5, 3, 1, 5, 5, 2, 4, 3, 4, 3, 3, 1, 5, 1, 1, 1, 2, 6, 1, 5, 4, 6, 5, 4, 1, 6, 1, 5, 5, 4, 4, 4, 2, 6, 4, 1, 1, 5, 5, 4, 2, 5, 4, 3, 2, 0, 3, 2, 2, 2, 4, 3, 5, 3, 6, 2, 4, 5, 0, 6, 0, 2, 1, 5, 4, 0, 4, 0, 2, 6, 4, 4, 2, 6, 2, 1, 0, 5, 2, 3, 3, 4, 4, 1, 2, 3, 2, 2, 0, 1, 6, 4, 3, 5, 2, 0, 6, 1, 2, 3, 5, 5, 3, 6, 1, 1, 4, 1, 2, 2, 1, 3, 4, 2, 3, 6, 2, 5, 1, 5, 1, 5, 1, 2, 0, 3, 5, 2, 1, 0, 4, 6, 6, 6, 1, 3, 0, 1, 5, 4, 6, 5, 3, 5, 5, 5, 0, 1, 1, 3, 0, 1, 0, 4, 0, 6, 2, 4, 3, 5, 1, 0, 5, 0, 3, 4, 1, 6, 6, 3, 6, 4, 4, 5, 2, 2, 1, 5, 5, 1, 4, 0, 4, 5, 4, 6, 4, 4, 5, 1, 5, 5, 4, 5, 4, 2, 4, 5, 4, 0, 3, 3, 2, 0, 2, 5, 5, 4, 1, 0, 3, 6, 0, 3, 6, 5, 4, 6, 4, 4, 4, 4, 6, 4, 2, 3, 2, 2, 2, 5, 4, 5, 6, 4, 2, 3, 0, 6, 6, 0, 4, 6, 6, 3, 2, 1, 6, 5, 2, 5, 1, 0, 0, 3, 5, 0, 6, 5, 4, 1, 2, 2, 2, 1, 0, 4, 5, 0, 2, 4, 

In [32]:
submission['topic_idx'] = pred

In [33]:
submission.head()

,index,topic_idx
0,45654,2
1,45655,3
2,45656,2
3,45657,2
4,45658,3


In [35]:
submission.to_csv("results/kobert.csv",index=False)